# Metaprogramação em Julia
Esse tutorial é baseado no Workshop em Metaprogramming de 2021 da JuliaCon ministrado por 
David P. Sanders. O material do workshop pode ser encontrado [neste repositório](https://github.com/dpsanders/Metaprogramming_JuliaCon_2021).


In [7]:
using Pkg
Pkg.activate(".")
using StatsBase: sample
using Random

  Activating project at `~/MEGA/EMAp/Julia_Tutorials/MetaProgramming`


## Introdução - Exemplo Inicial

Metaprogramação consiste em utilizar a própria linguagem de programação para escrever e manipular código dela mesma, e ainda
avaliar este código. Metaprogramação não é algo presente em toda linguagem. Felizmente, em Julia temos essa possibilidade,
o que nos permite gerar código de maneira bastante sofisticada.

Vamos começar com um exemplo bastante simples. Suponha que você tem uma lista de variáveis que recebem um valor aleatório.
Você então que descobrir qual dessas variáveis contém um certo valor. No código abaixo, nós amostramos sem reposição
os valores 1 até 5, e colocamos dentro das variáveis `a`, `b`, `c`, `d` e `e`. Asssim, de antemão, nós não sabemos
qual valor está contido em cada uma delas. Como podemos descobrir, por exemplo, qual variável contém o valor 1?

In [8]:
Random.seed!(7)
a,b,c,x,y = sample(1:5, 5, replace=false);

Para descobrir quem tem o valor 1, podemos ir printando o valor de cada uma junto com um `if`. Porém, percebemos
que teríamos que realizar um tipo de trabalho "manual". Por exemplo:

In [9]:
for (i,v) in enumerate([a,b,c,x,y])
    v == 1 ? println(i) : nothing
end

4


O código mostrou que a quarta variável é que contém o valor 1. Agora olhamos na nossa lista `[a,b,c,x,y]` e vemos que a quarta variável é `x`. Logo ela contém o valor 1. Veja que o trabalho "manual" foi que nós tivemos que olhar qual era a variável na lista. E se quisermos evitar esse trabalho "manual" de olhar na lista nós mesmos? Como "printar" o nome na variável na tela?
Bem, sem metaprogramção, uma solução seria fazer isso na "força bruta":

In [10]:
a == 1 ? print("a") : nothing
b == 1 ? print("b") : nothing
c == 1 ? print("c") : nothing
x == 1 ? print("x") : nothing
y == 1 ? print("y") : nothing

x

Conseguimos, mas tivemos que escrever um monte de linhas de código. Para cinco variáveis isso não é problema, mas se tivéssemos 
bem mais variáveis, isso se tornaria tedioso.
É nesse tipo de problema (e coisa bem mais complexas), que a metaprogramação nos ajuda. Vamos começar
usando o comando `names(Main)`. A função `name()` recebe o nome de um `module`, e rotorna todos os nomes sendo exportados por aquele
`module`. No caso, `Main` é o `module` padrão que estamos utilizando. O `Main`, por padrão, exporta `Base`, `Core` e `Main`, e além disso,
´ele exporta toda variável criada nele. Assim, ao usar `names(Main)`, vamos pegar todos os resultados a partir do quarto índice.

In [11]:
names(Main)[4:end]

5-element Vector{Symbol}:
 :a
 :b
 :c
 :x
 :y

Muito bem, conseguimos de maneira rápida o nome de nossas variáveis. Mas como fazemos para avaliar o valor em cada uma delas?
Novamente, sem metaprogramação, o `names(Main)` não será muito útil, porém, com metaprogramação, é bastante. Abaixo iremos apresentar
a solução, e na seção seguinte iremos explicar melhor o que foi feito e os básicos da metaprogramação em Julia.

In [14]:
for v in names(Main)[4:end]
    eval(v) == 1 ? println(v) : nothing
end

x


## Básicos da Metaprogramação

Vimos que o código anterior foi capaz de facilmente retornar a variável com o valor 1. Além disso, é facíl ver que ele funciona imediatamente
para casos envolvendo mais variáveis, sem precisar ser modificado. O que não é verdade para as outras duas soluções que apresentamos.
A nossa solução envolveu o uso de uma função chamada `eval`, qu é uma função do módulo `Core`, ou seja, `eval` é o mesmo que
`Core.eval`.
Como você já deve ter adivinhado, a função `eval` recebe uma expressão em Julia, e avalia essa expressão. Mas o que é uma "expressão"?

Bem, podemos pensar inicialmente que todo código em Julia é composto de símbolos (`Symbol`) e valores literais. A combinação de
símbolos com valores literais forma uma expressão (`Expr`). De forma simplista, uma expressão é formada por um símbolo pai (`head`)
seguido de argumentos, que podem ser outros símbolos, valores literais ou outras expressões ("it's all symbols and values all the way down").
O `head` determina o tipo da expressão, enquanto os demais argumentos representam o que de fato a expressão faz.

Vamos ver um exemplo:

é composto de dois valores literais (1 e 2),
e um símbolo (`+`). Essa mesma expressão pode ser escrita como `Expr()

In [27]:
1 + 2

3

A linha de código acima é interpretada em Julia como uma expressão com os argumentos `1`, `2` e `:+`, onde os dois primeiros são valores
literais e o último é um símbolo. Essa mesma expressão pode ser representada por `Expr(:call, :+, 1, 2)`. Ou seja, é sua `head` contém
o símbolo `:call`, representando justamente que essa expressão está sendo avaliada.

In [42]:
ex = Expr(:call, :+, 1, 2)

ex.head, ex.args

(:call, Any[:+, 1, 2])

Assim, uma vez que temos uma expressão, podemos passar utilizar a função `eval`, que ela irá
executar o que está descrito na expressão.

Como vimos no exemplo acima, a estrutura de uma expressão costuma ser mais complexa do que a forma "natural"
com a qual escrevemos código. Assim, se quisermos escrever código em Julia usando Julia, uma opção
mais simples seria escrever strings, para então torná-los expressões. E isso é exatamente o que faz a função `Meta.parse`.

In [45]:
ex_string = "1 + 2 / 2"
ex = Meta.parse(ex_string)

:(1 + 2 / 2)

Isso torna todo o processo de metaprogramação bem menos laborioso, pois sabemos como escrever código sem ser formato de expressão,
e deixamos para a `Meta.parse` o trabalho de converter "código natural" no formato de expressões.

Caso queiramos entender melhor como está estruturada uma expressão, podemos utilizar a função `dump()` ou a função `Meta.show_sexpr`.

In [47]:
dump(ex)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Int64 1
    3: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol /
        2: Int64 2
        3: Int64 2


In [48]:
Meta.show_sexpr(ex)

(:call, :+, 1, (:call, :/, 2, 2))

Usando essas funções, fica claro, por exemplo, que nosso código primeiro avalia a divisão (`2 / 2`), para depois realizar a soma. 
Assim, esses comando podem ser úteis caso você tenha dúvida de como Julia está processando uma linha de código específica.

## Outro Exemplo
Imagine que um colega seu possui um monte de scripts em Julia. O que você gostaria de fazer é rodar o código dele,
e sempre que uma variável do tipo `Importante` for gerada, você quer salvar o nome e valor dessa variável numa pasta.
Quais são os desafios? Bem, o primeiro dessafio é que ao menos que rodemos cada script, não vamos saber qual é a variável que é do tipo
`Importante`. Então, pra começar, precisamos rodar cada script. Além disso, precisamos saber o nome da variável, e então verificar se ela é
do tipo desejado... Não parece simples. Como metaprogramação nos ajudaria?

Fazer isso sem metaprogramção não parece ser trivial. Mas e com metaprogramação? Com metaprogramação conseguimos
rapidamente pensar em alumas maneiras de atacar esse problema que não envolve tentar entender o código
do seu amigo para descobrir em que momento ele declara variáveis do tipo `Importante`.

Por exemplo, uma estratégia seria avaliar quando uma linha de código está atribuindo valor para uma variável via `=`,
e então checar se essa variável retorna o tipo `Importante`. Se sim, nós então salvamos o nome e o valor dela. Esse problema
não é tão simples assim, por simplicidade, vamos supor que o código do seu amigo consiste em "uma linha, uma expressão", ou seja,
cada linha representa uma ação. Assim, podemos quebrar cada linha como uma expressão em Julia e avaliar a expressão.

In [49]:
codigo = """
#### Código do seu amigo
struct Importante valor::Real end
y = 0.4
x = 1 * y
x = Importante(10)
x.valor + y
z = Importante(20)
store = z.valor + y + x.valor
"""

module Amigo

export x
x = 1
end

store = []
for code in split(codigo,"\n")
    ex = Meta.parse(code)
    if ex !== nothing
        Core.eval(Amigo, ex)
        if String(ex.head) == "="
            if typeof(Core.eval(Amigo,ex.args[1])) <: Amigo.Importante
                push!(store,(ex.args[1],Core.eval(Amigo, ex.args[1]).valor))
            end
        end
    end
end

store

2-element Vector{Any}:
 (:x, 10)
 (:z, 20)

O código funciona! Mas você deve estar se perguntando o que esse `module Amigo` está fazendo? O seu proprósito é isolar 
o código que está rodando do seu amigo, com o código que estamos rodando. Por exemplo, o código do seu amigo
termina com a linha `store = z.valor + y + x.valor`. Ou seja, ele está definindo uma variável `store`, que tem o mesmo
nome da variável que estamos usando para rodar o nosso código. Se usássemos o `Core.eval(ex)`, sem o `Amigo`, isso
acabaria alterando o valor da nossa variável `store`, causando problemas.

Você pode ainda estar se perguntando, *Mas afinal isso tem realmente algum uso prático?!* E a reposta é **sim**.
Inclusive, o tipo de problema que resolvemos no exmemplo acima é baseado num problema real que tive que
resolver quando programando o pacote `NotebookToLaTeX.jl`. Esse pacote converte notebooks Jupyter e Pluto para arquivos em LaTeX.
Um dos desafios com notebooks Pluto é que eles são nada mais que scripts em Julia, isto é, arquivos `.jl`.
Diferente do Jupyter, o Pluto não guarda os plots gerados dentro dele, sendo assim necessário rodar o código para gerar as figuras.

Meu desafio então era descobrir em que momento um plot estava sendo gerado, para poder então salvar a figura em uma pasta para
que o arquivo LaTeX pudesse ler.
Similar ao que tivemos no exemplo acima, meu código rodava cada expressão do notebook e checava para ver se era do tipo adequado.
Foi inclusive resolvendo esse problema que a comunidade de Julia sugeriu o uso de `module` como maneira de evitar
variáveis trocando valores em lugares indesejados.

In [ ]:
x = 10
y = 5

@show y + x

In [ ]:
@macroexpand @show y + x

Veja que a expressão "quote" simboliza que essa expressão é código em Julia. Ou seja, poderíamos copiar e 
colar o que está dentro desse "quote" no nosso REPL. A segunda expressão estranha é esse comentário
"#= show.jl:955 =#". Neste caso, esse comentário está informando em que linha do source code está definida essa função.
Por fim, temos `var"#788#value"`. Isso é o nome da variável que o código criou. Esse nome é estranho
justamente com o propósito de evitar que já tenha essa variável no se código.

In [ ]:
code = Meta.parse("j = i^2")

In [ ]:
typeof(code)

In [ ]:
dump(code)

In [ ]:
code.args

In [ ]:
code.head

In [ ]:
dump(code.args[2])

In [ ]:
code2 = copy(code)
code2.args[1] = :k
code2.args[2].args[3] = 3
code2

Em Julia, temos o tipo `Symbol` que pode ser entendido como uma expressão antes de ser
avaliada. Por exemplo, caso escrevamos `x = 2`. Ao escrever `x`, o compilador
irá avaliar essa expressão, retornando `2`. Entretanto, as vezes queremos
nos referir não ao que está dentro de `x`, mas ao símbolo em si. Para isso,
usamos `:x`.

In [ ]:
Symbol("x") == :x

Da mesma forma, podemos falar sobre a expressão `x + y` usando `:(x + y)`. Considere a seguinte situação,
você está escrevendo uma série de comandos no REPL. Você percebe que declarou tantas variáveis que não sabe mais
qual delas está armazenando o valor `10`. Julia permite que você faça isso de dentro da própria linguagem.

In [ ]:
x = 1
y = 10
z = 2

for i in 'x':'z'
    v = Meta.parse("$i")
    if eval(v) == 10
        return v
    end
end

A função `eval` está justamente avaliando a expressão dentro de `v`. O que fizemos no código acima foi iterar pela
nossas variáveis nos referindo a elas pelo seu símbolo. Imagine agora que cada uma das letras do alfabeto
foi definida como uma variável e queremos descobrir novamente qual delas armazena `10`. Como faríamos
isso sem metaprogramação? Possívelmente teríamos que manualmente escrever algo do tipo
`for v in [a,b,c,...,z]`. Com metaprogramação, podemos simplesmente escrever o loop usando `'a':'z'`.

### Referências
* [Documentação de Julia sobre Metaprogramação](https://docs.julialang.org/en/v1/manual/metaprogramming/);